# Ingesting Prochaska+11 CGM dataset [v1.1]

In [1]:
# imports
from pkg_resources import resource_filename

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

import pyigm
from pyigm.cgm.cgmsurvey import CGMAbsSurvey

## Summary

In [2]:
ovi_file = resource_filename('pyigm', '/data/CGM/P11/lowovidat.fits')

In [3]:
ovidat = Table.read(ovi_file)

In [4]:
ovidat[0:3]

QSO,QSO_RA,QSO_DEC,QSO_ZEM,QSO_VMAG,QSO_UV,FLG_GAL,GAL_FIL,R_LIMIT,N_GAL [4],"COMPLETE [2,4]",FLG_FUSE,FUSE_EXP,FUSE_SNR,FLG_STIS,STIS_COMM [4],FLG_GHRS,GHRS_COMM [4]
str15,str15,str15,float32,float32,float32,int16,str58,float32,int32,int32,int16,float32,float32,int16,str60,int16,str60
Q0026+1259,00:29:13.8,+13:16:04,0.142,14.78,20.0,1,/u/xavier/LCO/OVI/FUSE/data/Q0026+1259/Q0026+1259_gal.fits,20.0,131 .. 47,22 .. 85,1,20000.0,7.0,0,..,1,G270M 5222s ..
TONS180,00:57:20.0,-22:22:56,0.06198,16.6,30.0,1,/u/xavier/LCO/OVI/FUSE/data/TONS180/TONS180_gal.fits,19.7,110 .. 4,15 .. 92,1,132453.0,15.0,2,G140M 7000s 15. ..,0,..
TONS210,01:21:51.5,-28:20:57,0.116,14.7,70.0,1,/u/xavier/LCO/OVI/FUSE/data/TONS210/TONS210_gal.fits,20.0,71 .. 5,6 .. 87,1,56500.0,20.0,2,E140M 22000s ..,0,..


In [5]:
qso_radec = SkyCoord(ra=ovidat['QSO_RA'], dec=ovidat['QSO_DEC'], unit=(u.hourangle, u.deg))

## Dwarfs

In [6]:
cgm_dwarf_file = resource_filename('pyigm','/data/CGM/P11/dwarf_galabs_strct.fits')

In [7]:
cgm_dwarfs = Table.read(cgm_dwarf_file)

In [8]:
cgm_dwarfs[0:3]

FIELD,ID,OBJ_ID,FLG_ANLY,FLG_SURVEY,OBJ_TYPE,MAG [10],MAGERR [10],FILTER [10],IMG_FIL [10],XYPIX [2],RA,DEC,AREA,STARGAL,GAL_TYPE,GAL_COEFF [10],Z,VCIRC,FSPEC_FIL [10],DRA,DDEC
str11,int32,str1,int16,int16,int16,float32,float32,str1,str27,float32,float64,float64,float32,float32,str5,float32,float64,float32,str29,float64,float64
Q0026+1259,1303,a,7,1,0,19.733 .. 0.0,0.125 .. 9.99,B ..,Images/Q0026+1259XB.fits ..,1014.76 .. 1034.29,7.28886499155,13.2746021629,10.43,0.18,Late,0.331349 .. 0.0,0.0329451337457,0.0,..,45.6990947951,0.0191390593786
TONS180,2295,a,7,1,0,18.923 .. 1.0,0.088 .. 0.05,B ..,Images/TONS180XB.fits ..,1318.89 .. 607.18,14.2667432785,-22.44755991,10.92,0.19,Late,-0.0115093 .. 0.0,0.0233643911779,0.0,..,154.07390626,0.0207054292147
PKS0405-123,90033,a,7,1,0,0.0 .. 0.44,0.0 .. 0.06,..,..,61.9512 .. -12.1839,61.9512481689,-12.1838884354,0.0,0.0,Late,0.0 .. 0.0,0.167,1.0,..,97.7760719383,0.0877822129587


### Funny columns -- Renaming

In [9]:
cgm_dwarfs.rename_column('DRA', 'rho(kpc)')
cgm_dwarfs.rename_column('DDEC', 'L(L*)')

#### MAG has HI
    2 == Ref
    3 == z
    4 == Lya EW
    5 == sigEW Lya
    8 == NHI
    9 == sigNHI

In [10]:
cgm_dwarfs[1]['FIELD']

'TONS180    '

In [11]:
cgm_dwarfs[1]['MAG']

array([  1.89230003e+01,   1.80701466e+01,   5.00000000e+00,
         2.34109219e-02,   2.22000000e+02,   2.90000000e+01,
         5.60000000e+01,   4.00000000e+00,   1.38000002e+01,
         1.00000000e+00], dtype=float32)

In [12]:
Lya_EW = cgm_dwarfs[1]['MAG'][4]
Lya_EW

222.0

#### MAGERR has OVI
    2 == Ref
    3 == z
    4 == EW 1031
    5 == sigEW 1031
    8 == NOVI    
    9 == sigNOVI

In [13]:
cgm_dwarfs[1]['MAGERR']

array([  8.79999995e-02,   6.49999976e-02,   4.00000000e+00,
         2.33999994e-02,   4.30000000e+01,   1.50000000e+01,
         3.00000000e+01,   0.00000000e+00,   1.34799995e+01,
         5.00000007e-02], dtype=float32)

#### Refs

In [14]:
refdict = {1: 'tripp08', 2: 'tc08a', 3: 'ds08', 4: 'dsr+06', 5: 'pss04', 6: 'cm09', 9: 'p+11'}

## Ingest
    python ingest_lit.py

## Read

In [15]:
p11_tarfile = resource_filename('pyigm','/data/CGM/P11/P11_sys.tar')

In [29]:
p11 = CGMAbsSurvey.from_tarball(p11_tarfile, chk_lowz=False, chk_z=False)

/home/xavier/local/Python/linetools/linetools/lists/linelist.py:402: RuntimeWarning: divide by zero encountered in log10
  self._extra_table['log(w*f)'] = np.log10(qm_strength)
/home/xavier/local/Python/linetools/linetools/spectralline.py:214: UserWarning: Redshift not input.  Setting to 0 for zLimits
  warnings.warn("Redshift not input.  Setting to 0 for zLimits")


Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Skipping a likely folder: CGM_JSON


In [30]:
p11

<CGM_Survey:  nsys=54, ref=>
<IGMSystem: IGMSystem 00:29:13.8019 13:16:04.0065, zabs=0.0329451, Z/H=0, NHI=0>
<IGMSystem: IGMSystem 00:29:13.8019 13:16:04.0065, zabs=0.039311, Z/H=0, NHI=0>
<IGMSystem: IGMSystem 00:57:20.0009 -22:22:55.9961, zabs=0.0233644, Z/H=0, NHI=13.8>
<IGMSystem: IGMSystem 00:57:20.0009 -22:22:55.9961, zabs=0.045619, Z/H=0, NHI=13.8>
<IGMSystem: IGMSystem 03:11:55.1958 -76:51:51.0087, zabs=0.202643, Z/H=0, NHI=15.14>
<IGMSystem: IGMSystem 03:11:55.1958 -76:51:51.0087, zabs=0.0593531, Z/H=0, NHI=13.53>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133, zabs=0.0964516, Z/H=0, NHI=14.64>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133, zabs=0.297609, Z/H=0, NHI=13.85>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133, zabs=0.352, Z/H=0, NHI=14.04>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133, zabs=0.153212, Z/H=0, NHI=13.46>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133, zabs=0.203022, Z/H=0, NHI=13>
<IGMSystem: IGMSystem 04:07:48.4013 -12:11:37.0133

In [31]:
p11.rho

<Quantity [  46.32007042, 231.08717107, 153.91919088, 281.05023842,
             34.78581646, 241.08235808, 271.56656557, 263.85542101,
            170.30403727, 193.11905036, 277.85573984, 228.92416305,
            101.21597012, 122.33510368, 181.21741485,  78.67047199,
            306.79467169, 159.32049028, 133.01256117, 221.58366503,
            137.97841172, 151.94838708,  72.60291124, 182.78669925,
            138.10197377,  94.40182666, 106.07146536,  92.90730056,
             37.14686961,  85.54748551,  41.25864481, 260.27253479,
            120.93128195, 243.45856691, 225.98146589,  69.86583806,
            227.78876094,  88.46379097,  99.49390855, 274.8521744 ,
            162.60742332, 309.02130732, 176.70156002, 282.59706888,
            293.98085988, 184.38541882, 228.32783121, 305.40178171,
            266.93146969, 292.71386558, 236.98360734,  35.06117805,
            271.77462491, 108.99879108] kpc>

In [32]:
p11.NHI

array([  0.        ,   0.        ,  13.80000019,  13.80000019,
        15.14000034,  13.52999973,  14.64000034,  13.85000038,
        14.03999996,  13.46000004,  13.        ,  15.14000034,
        15.47000027,  15.47000027,   0.        ,  13.89999962,
        14.        ,  14.43000031,  13.64000034,  13.64000034,
        14.35000038,  15.72999954,  15.72999954,  15.67000008,
        15.67000008,  14.77999973,  13.93999958,  19.29999924,
        13.10000038,  15.38000011,   0.        ,   0.        ,
         0.        ,   0.        ,  15.01000023,  15.35000038,
        14.82999992,  15.28999996,   0.        ,   0.        ,
        13.10000038,  13.67000008,   0.        ,   0.        ,
         0.        ,   0.        ,  14.30000019,  14.07999992,
        13.06000042,  15.10999966,  14.28999996,  15.10999966,
        14.07999992,  13.56000042])

## Lya EW

In [33]:
p11.cgm_abs[2].igm_sys._components
#print(icgm.igm_sys)
#icgm.igm_sys._components

[<AbsComponent: 00:57:20 -22:22:56, Name=OVI_z0.02340, Zion=(8,6), Ej=0 1 / cm, z=0.0234, vlim=-400 km / s,400 km / s, logN=13.48, sig_logN=0.05000000074505806, flag_N=1>,
 <AbsComponent: 00:57:20 -22:22:56, Name=HI_z0.02341, Zion=(1,1), Ej=0 1 / cm, z=0.0234109, vlim=-400 km / s,400 km / s, logN=13.8, sig_logN=1.0, flag_N=1>]

In [35]:
p11.cgm_abs[2].igm_sys._components[1]._abslines

[<AbsLine: HI 1215, z=0.0234 wrest=1215.6700 Angstrom, f=0.4164>]

In [37]:
Lya = p11.cgm_abs[2].igm_sys._components[1]._abslines[0]
Lya.attrib

{'EW': <Quantity 0.222 Angstrom>,
 'N': <Quantity 0.0 1 / cm2>,
 'Ref': 5,
 'b': <Quantity 0.0 km / s>,
 'coord': <SkyCoord (ICRS): (ra, dec) in deg
     ( 0.,  0.)>,
 'flag_EW': 1,
 'flag_N': 0,
 'logN': 0.0,
 'sig_EW': <Quantity 0.029 Angstrom>,
 'sig_N': <Quantity 0.0 1 / cm2>,
 'sig_b': <Quantity 0.0 km / s>,
 'sig_logN': 0.0,
 'sig_v': <Quantity 0.0 km / s>,
 'sig_z': 0.0,
 'v': <Quantity 0.0 km / s>}

### Table

In [38]:
Lya_tbl = p11.ion_tbl((1,1))

In [39]:
Lya_tbl

name,Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN,cgm_name,rho_impact
,,,,1 / cm,,km / s,km / s,,,,,kpc
str32,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,str18,float64
J002913.8+131604_z0.033,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,J0029+1316_291_70,46.3200704229
J002913.8+131604_z0.039,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,J0029+1316_4_294,231.08717107
J005720-222256_z0.023,1,1,0,0.0,0.0234054606408,-398.400194945,401.599805055,1,13.8000001907,1.0,J0057-2222_223_323,153.919190876
J005720-222256_z0.046,1,1,0,0.0,0.0456186737865,-394.646203256,405.353796744,1,13.8000001907,1.0,J0057-2222_329_310,281.050238424
J031155.2-765151_z0.203,1,1,0,0.0,0.202704995871,-388.782692027,411.217307973,1,15.1400003433,0.164999991655,J0311-7651_117_10,34.7858164592
J031155.2-765151_z0.059,1,1,0,0.0,0.0594599992037,-400.0,400.0,1,13.529999733,0.0299999993294,J0311-7651_174_208,241.082358081
J040748.4-121137_z0.096,1,1,0,0.0,0.0965700000525,-400.0,400.0,1,14.6400003433,0.114999994636,J0407-1211_109_150,271.566565575
J040748.4-121137_z0.298,1,1,0,0.0,0.297625005245,-394.223250089,405.776749911,1,13.8500003815,0.125,J0407-1211_145_59,263.855421013
